In [ ]:
import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
DNF_STATUSES = ['Collision damage', 'Hydraulics', 'Radiator', 'Collision', 'Retired', 'Did not start', 'Mechanical', 'Electronics']

In [ ]:
def compute_synergy_score(metrics: dict) -> float:
    score = (
        -metrics['Teammate_delta'] * 2.0 +  # negative delta = faster than teammate
        -metrics['Lap_stdev'] * 1.5 +     # lower std dev = more consistent
        -metrics['Avg_Q'] * 0.5 +         # higher qual = better driver performance
        -metrics['Avg_R'] * 1.0 +         # higher race position = better result
        -metrics['DNFRate'] * 3.0           # fewer DNFs = more reliable
    )
    return score

In [ ]:
def compute_average_laptime(laps, driver_list):

    driver_averages = []

    for driver in driver_list:
        driver_laps = laps.pick_drivers(driver)
        driver_laps['LapTimeSeconds'] = driver_laps['LapTime'].dt.total_seconds()
        clean_laps = driver_laps[
        driver_laps['PitInTime'].isnull() & 
        driver_laps['PitOutTime'].isnull() & 
        driver_laps['IsAccurate'] & 
        driver_laps['LapTime'].notnull()
        ]['LapTimeSeconds']

        if clean_laps.empty:
            return [0 ,0]
        laps_avg = clean_laps.mean()
        driver_averages.append(laps_avg)
    return driver_averages

In [ ]:
def compute_lap_deviation(laps, driver):
    # Get laps for the selected driver
    driver_laps = laps.pick_drivers(driver)

    # Convert laptime from Timedelta to seconds
    driver_laps['LapTimeSeconds'] = driver_laps['LapTime'].dt.total_seconds()
    # Filter valid laps only (not in lap, out lap, and laptimes must be accurate)
    clean_laps = driver_laps[
        driver_laps['PitInTime'].isnull() & 
        driver_laps['PitOutTime'].isnull() & 
        driver_laps['IsAccurate'] & 
        driver_laps['LapTime'].notnull()
    ]['LapTimeSeconds']
    if clean_laps.empty:
        return 0
    # Perform simple standard deviation
    laps_std = clean_laps.std()
    return laps_std

In [ ]:
drivername = 'Lewis Hamilton'
driver_synergies = []

for year in range(2021, 2025):
    events = fastf1.get_event_schedule(year)
    for race_index, event in events.iterrows():
        if event['EventFormat'] != 'testing':
            race = event.get_race()
            race.load()
            race_results = race.results
            driver_res = race_results.loc[race_results['FullName'] == drivername]
            if not driver_res.empty:

                stats = {
                    'Teammate_delta' : 0,
                    'Lap_stdev': 0,
                    'Avg_Q': 0,
                    'Avg_R': 0,
                    'DNFRate': 0
                }

                driver_res = driver_res.iloc[0]
                team_id = driver_res['TeamId']

                teammate_res = race_results.loc[(race_results['TeamId'] == team_id) & (race_results['FullName'] != drivername)]
                if teammate_res.empty:
                    continue
                teammate_res = teammate_res.iloc[0]

                teammate_abbr = teammate_res['Abbreviation']
                driver_abbr = driver_res['Abbreviation']
                race_laps = race.laps
                driver_avg, teammate_avg = compute_average_laptime(race_laps, [driver_abbr, teammate_abbr])
                driver_stdev = compute_lap_deviation(race_laps, driver_abbr)
                stats['Lap_stdev'] = driver_stdev
                race_delta = driver_avg - teammate_avg
                if race_delta != 0:
                    stats['Teammate_delta'] = race_delta
                stats['Avg_Q'] = driver_res['GridPosition']
                stats['Avg_R'] = driver_res['Position']
                if driver_res['Status'] in DNF_STATUSES: stats['DNFRate'] = 1
                else: stats['DNFRate'] = 0

                synergy = compute_synergy_score(stats)
                stats['Season'] = year
                stats['Round'] = race_index
                stats['Synergy'] = synergy
                driver_synergies.append(stats)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df = pd.DataFrame(driver_synergies)
plotting_df = df[['Avg_R', 'Synergy']]
scaler = MinMaxScaler(feature_range=(0, 20))
plotting_df['Synergy'] = scaler.fit_transform(plotting_df[['Synergy']])

fig, ax1 = plt.subplots(figsize=(10, 5))

# Plot Synergy Score on primary y-axis
ax1.plot(df.index, plotting_df['Synergy'], color='tab:blue', marker='o', label='Synergy Score')
ax1.set_xlabel('Race')
ax1.set_ylabel('Synergy Score', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Create secondary y-axis for Race Result
ax2 = ax1.twinx()
ax2.plot(df.index, plotting_df['Avg_R'], color='tab:red', linestyle='--', marker='s', label='Race Result')
ax2.set_ylabel('Race Result (lower is better)', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
ax2.invert_yaxis()  # Lower result (e.g. 1st place) is higher on the chart

# Title and layout
plt.title('Synergy Score vs Race Result per Race')
fig.tight_layout()
plt.grid(True)
plt.show()